In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense,Conv1D
from sklearn.metrics import f1_score,roc_auc_score,confusion_matrix


#Data Processing

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
data.drop(["id","Unnamed: 32"],axis=1,inplace=True)

In [ ]:
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
X,y = data.drop("diagnosis",axis=1),data["diagnosis"]

In [ ]:
y = (y=='M').astype('int')

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,stratify=y)

In [ ]:
top_features_corr = ['fractal_dimension_se', 'compactness_worst', 'texture_worst', 'symmetry_se', 'perimeter_mean', 'perimeter_se', 'texture_se', 'radius_worst', 'concave points_worst', 'fractal_dimension_mean']

top_features_chi2 = ['perimeter_worst', 'perimeter_mean', 'area_mean', 'area_worst', 'area_se', 'radius_worst', 'radius_mean', 'perimeter_se', 'texture_worst', 'texture_mean']

top_features_rfe = ['radius_mean', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'symmetry_worst', 'area_se', 'perimeter_se', 'compactness_worst', 'texture_worst', 'radius_se']

top_features_rfecv = ['radius_mean', 'concave points_worst', 'concavity_worst', 'compactness_worst', 'smoothness_worst', 'area_worst', 'perimeter_worst', 'texture_worst', 'radius_worst', 'concavity_se']

top_features_tbf = ['area_mean', 'concavity_mean', 'concave points_mean', 'area_se', 'radius_worst', 'perimeter_worst', 'area_worst', 'concavity_worst', 'concave points_worst']

top_features_vbf = ['perimeter_worst', 'radius_worst', 'area_worst', 'concave points_worst', 'concave points_mean', 'perimeter_mean', 'radius_mean', 'concavity_mean', 'area_mean', 'concavity_worst']

In [ ]:
overall_top_features = ['perimeter_worst','radius_worst','area_worst','concave points_worst','texture_worst','radius_mean','compactness_worst','concavity_worst','perimeter_mean','area_mean']

In [ ]:
# M -> 1 B->0

In [ ]:
All_importent_Features = [top_features_corr,top_features_chi2,top_features_rfe,top_features_rfecv,top_features_tbf,top_features_vbf,overall_top_features]

In [ ]:
def classification_different_featureSelection_methods(clf,X_train,y_train,X_test,y_test,featureMethod=None):
    if featureMethod != None:
        X_train = X_train[featureMethod]
        X_test = X_test[featureMethod]
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = clf.score(X_test,y_test)
    f1 = f1_score(y_test,y_pred)
    auc = roc_auc_score(y_test,y_pred)
    confusionMatrix = confusion_matrix(y_test,y_pred)

    return confusionMatrix,accuracy,f1,auc 

# Machine Learning Models

## LogisticRegression

In [ ]:
lr = LogisticRegression()
lr_outputs = [] 
for feature_selection in All_importent_Features:
    output = classification_different_featureSelection_methods(lr,X_train,y_train,X_test,y_test,featureMethod=feature_selection)
    lr_outputs.append(output)

In [ ]:
for i in lr_outputs:
    print(i)

(array([[86,  4],
       [ 4, 49]]), 0.9440559440559441, 0.9245283018867925, 0.940041928721174)
(array([[85,  5],
       [ 3, 50]]), 0.9440559440559441, 0.9259259259259259, 0.9439203354297694)
(array([[86,  4],
       [ 4, 49]]), 0.9440559440559441, 0.9245283018867925, 0.940041928721174)
(array([[87,  3],
       [ 3, 50]]), 0.958041958041958, 0.9433962264150944, 0.9550314465408805)
(array([[87,  3],
       [ 5, 48]]), 0.9440559440559441, 0.923076923076923, 0.9361635220125786)
(array([[88,  2],
       [ 6, 47]]), 0.9440559440559441, 0.9215686274509803, 0.9322851153039832)
(array([[87,  3],
       [ 3, 50]]), 0.958041958041958, 0.9433962264150944, 0.9550314465408805)


## Linear SVC

In [ ]:
li_svc = LinearSVC()
li_outputs = [] 
for feature_selection in All_importent_Features:
    output = classification_different_featureSelection_methods(li_svc,X_train,y_train,X_test,y_test,featureMethod=feature_selection)
    li_outputs.append(output)

In [ ]:
for i in li_outputs:
    print(i)

(array([[90,  0],
       [20, 33]]), 0.8601398601398601, 0.7674418604651162, 0.8113207547169812)
(array([[83,  7],
       [ 5, 48]]), 0.916083916083916, 0.8888888888888888, 0.9139412997903564)
(array([[88,  2],
       [15, 38]]), 0.8811188811188811, 0.8172043010752688, 0.8473794549266246)
(array([[90,  0],
       [23, 30]]), 0.8391608391608392, 0.7228915662650602, 0.7830188679245282)
(array([[90,  0],
       [15, 38]]), 0.8951048951048951, 0.8351648351648352, 0.8584905660377358)
(array([[90,  0],
       [14, 39]]), 0.9020979020979021, 0.8478260869565216, 0.8679245283018868)
(array([[84,  6],
       [ 7, 46]]), 0.9090909090909091, 0.8761904761904762, 0.9006289308176101)


## Random Forest

In [ ]:
RF = RandomForestClassifier()
rf_outputs = [] 
for feature_selection in All_importent_Features:
    output = classification_different_featureSelection_methods(RF,X_train,y_train,X_test,y_test,featureMethod=feature_selection)
    rf_outputs.append(output)

In [ ]:
for i in rf_outputs:
    print(i)

(array([[84,  6],
       [ 3, 50]]), 0.9370629370629371, 0.9174311926605505, 0.9383647798742139)
(array([[86,  4],
       [ 6, 47]]), 0.9300699300699301, 0.9038461538461539, 0.9211740041928722)
(array([[88,  2],
       [ 2, 51]]), 0.972027972027972, 0.9622641509433962, 0.9700209643605869)
(array([[86,  4],
       [ 3, 50]]), 0.951048951048951, 0.9345794392523364, 0.949475890985325)
(array([[86,  4],
       [ 4, 49]]), 0.9440559440559441, 0.9245283018867925, 0.940041928721174)
(array([[85,  5],
       [ 4, 49]]), 0.9370629370629371, 0.9158878504672898, 0.9344863731656184)
(array([[85,  5],
       [ 3, 50]]), 0.9440559440559441, 0.9259259259259259, 0.9439203354297694)


## XGBoost

In [ ]:
xg = XGBClassifier()
xg_outputs = [] 
for feature_selection in All_importent_Features:
    output = classification_different_featureSelection_methods(xg,X_train,y_train,X_test,y_test,featureMethod=feature_selection)
    xg_outputs.append(output)

In [ ]:
for i in xg_outputs:
    print(i)

(array([[84,  6],
       [ 3, 50]]), 0.9370629370629371, 0.9174311926605505, 0.9383647798742139)
(array([[87,  3],
       [ 5, 48]]), 0.9440559440559441, 0.923076923076923, 0.9361635220125786)
(array([[88,  2],
       [ 2, 51]]), 0.972027972027972, 0.9622641509433962, 0.9700209643605869)
(array([[86,  4],
       [ 2, 51]]), 0.958041958041958, 0.9444444444444444, 0.958909853249476)
(array([[85,  5],
       [ 3, 50]]), 0.9440559440559441, 0.9259259259259259, 0.9439203354297694)
(array([[84,  6],
       [ 4, 49]]), 0.9300699300699301, 0.9074074074074073, 0.9289308176100629)
(array([[85,  5],
       [ 2, 51]]), 0.951048951048951, 0.9357798165137615, 0.9533542976939203)


# Deep Learning Models

In [ ]:
def DL_model_class(modelPath,X,y):
    model = tf.keras.models.load_model(modelPath)
    y_pred = model.predict(X)
    y_pred = (y_pred>0.5).astype('int')
    
    accuracy = model.evaluate(X,y,verbose=False)[1]
    f1 = f1_score(y,y_pred)
    auc = roc_auc_score(y,y_pred)
    confusionMatrix = confusion_matrix(y,y_pred)

    return confusionMatrix,accuracy,f1,auc 

## ANN model

In [ ]:
input_layer = tf.keras.Input(shape=(30,))
x = Dense(17,activation='relu')(input_layer)
x = Dense(21,activation='relu')(x)
x = Dense(19,activation='relu')(x)
x = Dense(7,activation='relu')(x)
output_layer = Dense(1,activation='sigmoid')(x)

In [ ]:
ANN_Model = tf.keras.Model(inputs=[input_layer],outputs=[output_layer])
ANN_Model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
ANN_Model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=tf.keras.callbacks.ModelCheckpoint('bestmodel/ANN',save_best_only=True))

Epoch 1/100
14/14 [==============================] - 0s 9ms/step - loss: 0.1377 - accuracy: 0.9507 - val_loss: 0.2315 - val_accuracy: 0.9091
INFO:tensorflow:Assets written to: bestmodel/ANN/assets
Epoch 2/100
14/14 [==============================] - 0s 3ms/step - loss: 0.1552 - accuracy: 0.9343 - val_loss: 0.2141 - val_accuracy: 0.9091
INFO:tensorflow:Assets written to: bestmodel/ANN/assets
Epoch 3/100
14/14 [==============================] - 0s 3ms/step - loss: 0.1305 - accuracy: 0.9460 - val_loss: 0.2083 - val_accuracy: 0.9091
INFO:tensorflow:Assets written to: bestmodel/ANN/assets
Epoch 4/100
14/14 [==============================] - 0s 4ms/step - loss: 0.1437 - accuracy: 0.9343 - val_loss: 0.2519 - val_accuracy: 0.9021
Epoch 5/100
14/14 [==============================] - 0s 4ms/step - loss: 0.1340 - accuracy: 0.9554 - val_loss: 0.1737 - val_accuracy: 0.9231
INFO:tensorflow:Assets written to: bestmodel/ANN/assets
Epoch 6/100
14/14 [==============================] - 0s 5ms/step - loss

ANN Model Output-

In [ ]:
DL_model_class('bestmodel1/ANN',X_test,y_test)

(array([[87,  3],
        [ 4, 49]]), 0.9510489702224731, 0.9333333333333333, 0.9455974842767296)

## CNN model

In [ ]:
input_layer = tf.keras.Input(shape=(30,1))
x = Conv1D(12,kernel_size=3)(input_layer)
x = Conv1D(5,kernel_size=3)(x)
x = Conv1D(9,kernel_size=3)(x)
x = tf.keras.layers.Flatten()(x)
x = Dense(7,activation='relu')(x)
output_layer = Dense(1,activation='sigmoid')(x)

In [ ]:
CNN_model = tf.keras.Model(inputs=[input_layer],outputs=[output_layer])

In [ ]:
CNN_model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
CNN_model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=tf.keras.callbacks.ModelCheckpoint('bestmodel/CNN/',save_best_only=True))

Epoch 1/100
14/14 [==============================] - 0s 12ms/step - loss: 0.1550 - accuracy: 0.9296 - val_loss: 0.1973 - val_accuracy: 0.9161
INFO:tensorflow:Assets written to: bestmodel/CNN/assets
Epoch 2/100
14/14 [==============================] - 0s 6ms/step - loss: 0.1702 - accuracy: 0.9272 - val_loss: 0.1881 - val_accuracy: 0.9231
INFO:tensorflow:Assets written to: bestmodel/CNN/assets
Epoch 3/100
14/14 [==============================] - 0s 6ms/step - loss: 0.1703 - accuracy: 0.9225 - val_loss: 0.2194 - val_accuracy: 0.9161
Epoch 4/100
14/14 [==============================] - 0s 5ms/step - loss: 0.1569 - accuracy: 0.9366 - val_loss: 0.3038 - val_accuracy: 0.8951
Epoch 5/100
14/14 [==============================] - 0s 5ms/step - loss: 0.1606 - accuracy: 0.9319 - val_loss: 0.1881 - val_accuracy: 0.9161
INFO:tensorflow:Assets written to: bestmodel/CNN/assets
Epoch 6/100
14/14 [==============================] - 0s 5ms/step - loss: 0.1295 - accuracy: 0.9390 - val_loss: 0.2147 - val_ac

CNN Model Output-

In [ ]:
DL_model_class('bestmodel/CNN',X_test,y_test)

(array([[86,  4],
        [ 6, 47]]), 0.9300699234008789, 0.9038461538461539, 0.9211740041928722)